In [ ]:
def visible_sc_check(s1, t_i_part, t_cyl_diag_raw, t_1_top_init, d_opt, print_inner = False):
    #The partition t_partition and t_cyl_diag are already taken to be filtered so that they line up
    width = 2*d_opt
    length = len(t_i_part)
    t_cyl_diag = [0,0]
    t_cyl_diag[0] = t_cyl_diag_raw[0]
    t_cyl_diag[1] = t_cyl_diag_raw[1][::-1]
    t_1_top_index = t_cyl_diag[1].index(t_cyl_diag[0][0])
    t_i_top = tuple([t_cyl_diag[1][(i+t_1_top_index)%length] for i in range(length)])
    if print_inner:
        print(t_i_top)
    t_i_top_rel_bot_indices = tuple([t_cyl_diag[0].index(i) for i in t_i_top])
    if print_inner:
        print(t_i_top_rel_bot_indices)
    t_i_top_part = [t_i_part[i] for i in t_i_top_rel_bot_indices]
    if print_inner:
        print(t_i_top_part)
    t_i_top_sc_start_pos = tuple(sorted([(t_i_top[i], (2*s1+t_1_top_init+sum(t_i_top_part[:i]))%width) 
                                         for i in range(length)], key=lambda x: x[1]))
    #t_i_top_sc_length is the bottom cylinder the second coordinate is the starting position of 
    #saddle connection t_i
    if print_inner:
        print(t_i_top_sc_start_pos)
    t_i_sc_length_shift = tuple([sum(t_i_part[:i]) for i in range(length)])
    if print_inner:
        print(t_i_sc_length_shift)
    for i in enumerate(t_i_sc_length_shift):
        t_i = t_cyl_diag[0][i[0]]
        if print_inner:
            print('t_i', t_i)
        t_i_length = t_i_part[i[0]]
        if print_inner:
            print('t_i_length', t_i_length)
        t_i_orig_start = [t_i_tup for t_i_tup in t_i_top_sc_start_pos if t_i_tup[0] == t_i][0]
        if print_inner:
            print('t_i_orig_start', t_i_orig_start)
        t_i_new_start = (t_i_orig_start[1] + t_i_sc_length_shift[i[0]])%width
        if print_inner:
            print('t_i_new_start', t_i_new_start)
        if t_i_new_start < 2*s1 or t_i_new_start > width - 2*t_i_length:
            if print_inner:
                print('False', 2*s1, width - 2*t_i_length)
            return False
    return True

def visible_top_init_check(s1, t_i_part, t_cyl_diag, d_opt):
    width = 2*d_opt
    t_1_top_init_max = width-2*s1
    total_t_1_top_init = []
    for t_1_top_init in range(0,t_1_top_init_max+1,2):
        if visible_sc_check(s1, t_i_part, t_cyl_diag, t_1_top_init, d_opt):
            total_t_1_top_init.append(t_1_top_init)
    if len(total_t_1_top_init) > 0:
        return (s1, t_i_part, t_cyl_diag, total_t_1_top_init)
    return []

def align_list_visible_sc_check(s1, align_list, d_opt):
    width = 2*d_opt
    t_1_max = width-2*s1
    total = []
    count = 0
    for i in align_list:
        count += 1
        t_i_part = i[0]
        t_1 = t_i_part[0]
        if t_1 <= t_1_max:
            for t_cyl_diag in i[1]:
                admissible = visible_top_init_check(s1, t_i_part, t_cyl_diag, d_opt)
                if len(admissible) > 0:
                    total.append(admissible)
        if count%10000==0:
            print(count,i)
    return total

def align_list_visible_sc_check_write_file(window_range, align_list, d_opt, filename_root):
    for window in window_range:
        filename = filename_root + '_' + str(window)
        align_list_visible = align_list_visible_sc_check(window, align_list, d_opt)
        with open(filename, 'w') as file:
            file.write(str(align_list_visible))
    print(filename + ' written')
    return filename + ' written'

In [5]:
def construct_all_align_list_files(window_range, t_range, align_list, d_opt, 
                                   filename_root_dir, filename_align_root, filename_visible_root):
    for t in t_range:
        align_list_filename = filename_root_dir + filename_align_root + str(t)
        with open(align_list_filename, 'r') as file:
            align_list = eval_align_list(file.read())
            print(align_list_filename + ' read')
            print(str(len(align_list)) + ' configurations to search')
        filename_root2 = str(t) + '_visible'
        filename_root = filename_root_dir + filename_visible_root + filename_root2
        #Recall H(1,1): s_0 \in {9, 11, 13, 15}
        align_list_visible_sc_check_write_file(window_range, align_list, filename_root)
    return 'done'

def align_list_visible_evaluate_list(window_range, t_range, d_opt):
    width = 2*d_opt
    eval_list = []
    for t in t_range:
        for s in window_range:
            if 2*t+2*s <= width:
            #This checks to make sure that the Window Lemma condition is met
                eval_list += [(t,s)]
    return eval_list

def align_visible_to_make_list(window_range, t_range, d_opt, filename_root):
    available_list = []
    total_possibilities = align_list_visible_evaluate_list(window_range, t_range, d_opt)
    for eval_tuple in total_possibilities:
        (t,s) = eval_tuple
        filename_root2 = str(t) + '_visible_' + str(s)
        filename = filename_root + filename_root2
        try:
            with open(filename, 'r') as file:
                align_list_visible_string = file.read()
                part_list_not_exists = False
#                print(part_list_not_exists)
        except:
            part_list_not_exists = True
        if part_list_not_exists:
            available_list += [eval_tuple]
    return available_list

def generate_align_list_visible(item, d_opt, file_name_dir, align_list_filename_root, visible_list_filename_root):
    (s,t) = item
    align_list_filename = file_name_dir + align_list_filename_root + str(s)
    with open(align_list_filename, 'r') as file:
        align_list = eval_align_list(file.read())
        print(align_list_filename + ' read')
        print(str(len(align_list)) + ' configurations to search')
    filename_root1 = file_name_dir + visible_list_filename_root
    filename_root2 = str(s) + '_visible'
    filename_root = filename_root1 + filename_root2
    #Recall H(1,1,1,1): 10 \leq t_0 \leq 25
    return align_list_visible_sc_check_write_file([t], align_list, d_opt, filename_root)

def generate_all_align_list_visible_files(window_range, t_range, d_opt, 
                                          file_name_dir, align_list_filename_root, visible_list_filename_root):
    total_list = align_list_visible_evaluate_list(window_range, t_range, d_opt)
    print('total_list generated')
    for item in total_list:
        generate_align_list_visible(item, d_opt, file_name_dir, align_list_filename_root, visible_list_filename_root)
    return 'list completed'

In [25]:
def print_pool_instructions(display = False):
    if not display:
        return
    print('Instructions for using Pool: \n \n' + 'Define the following function: \n')
    print('def generate_align_list_visible_pool_TAG(item): \n    return ' 
          + 'generate_align_list_visible_pre(item, file_name_dir, align_list_filename_root, visible_list_filename_root)')
    print('\n')
    print('if False: \n    pool = Pool(processes=2) \n' + ' '*4
          + 'pool.map(generate_align_list_visible_pool_TAG, align_visible_to_make_list(' 
          + 'window_range, t_range, \n' + ' '*78 + 'd_opt, filename_root) \n')
    print('Recall: filename_root for align_visible_to_make_list contains //align_list_visible//')
    return